# Fit a Gaussian Process to AirBnB data, and plot it.

Author: Kui Tang

Final project for Statistical Graphics and Communication

## 0. 90% of Data Science is Getting and Cleaning the Data...

The files in http://insideairbnb.com/get-the-data.html under 'New York City' are saved under `data/`.

The `data/nybb_14aav.zip` file is downloaded from http://www.nyc.gov/html/dcp/download/bytes/nybb_14aav.zip.

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import statsmodels.api as sm
import seaborn as sns
import GPy

sns.set_style('white')


from IPython.core.display import HTML

import matplotlib.pyplot as plt
%matplotlib inline

RUN_DEAD_CODE = False

listings_df = pd.read_csv('data/listings.csv.gz')
calendar_df = pd.read_csv('data/calendar.csv.gz')
neighbourhoods_df = pd.read_csv('data/neighbourhoods.csv')

boros_gdf = gpd.GeoDataFrame.from_file('data/nybb_14a_av/nybb.shp')
neighbourhoods_gdf = gpd.GeoDataFrame.from_file('data/neighbourhoods.geojson')

In [ ]:
def clean_currency(x):
    x = str(x)
    if x[0] == '$':
        x = x[1:]
    x = x.replace(',', '')
    
    return float(x)

def clean_bool(x):
    return 1.0 if x == 't' else 0.0

def clean_percent(x):
    if np.isnan(x):
        return x
        
    x = str(x)
    if x[-1] == '%':
        x[-1] = ''
    
    return float(x) / 100.0
    
calendar_df['price'] = calendar_df['price'].apply(clean_currency)    
calendar_df['available'] = calendar_df['available'].apply(clean_bool)
calendar_df['date'] = pd.to_datetime(calendar_df.date)

cols_listings_currency = [u'price', u'weekly_price', u'monthly_price', u'security_deposit', u'cleaning_fee', u'extra_people']
for c in cols_listings_currency:
    listings_df[c] = listings_df[c].apply(clean_currency)
    
cols_listings_bool = ['is_location_exact', 'instant_bookable', 'host_identity_verified', 'host_is_superhost', 'host_has_profile_pic']
for c in cols_listings_bool:
    listings_df[c] = listings_df[c].apply(clean_bool)
    
cols_listings_cat = ['property_type', 'room_type', 'bed_type', 'cancellation_policy']
for c in cols_listings_cat:
    listings_df[c] = listings_df[c].astype('category')
    

In [ ]:
def clean_percent(x):
    x = str(x)
    if x == 'nan':
        return x
    
    if x[-1] == '%':
        x = x[:-1]
    
    return float(x) / 100.0

listings_df['host_response_rate'].apply(clean_percent)

Eighty-eight percent of the lat-long locations are exact, so we can reasonably plot them.

In [ ]:
listings_df['extra_people']

In [ ]:
listings_df['is_location_exact'].mean()

In [ ]:
with pd.option_context('display.max_rows', 10000):
    print listings_df.dtypes

In [ ]:
listings_df['availability_30']

In [ ]:
calendar_df

I'm not sure what this means. The spaces are never available to book? Everything's been booked up? At any rate, it should never be just one value. This must be a data error.

In [ ]:
calendar_df['available'].mean()

## 1. First Plot: Prices vs Location and Time

After viewing the results here, think of some covariates you could add to the Gaussian process.

### 1.1 Join calender and listing tables to obtain daily prices.

Some listings (a bit over 1%) show a zero for guests included! I really don't know how to deal with this other than to throw it out.

In [ ]:
cols_listings_geo = ['latitude', 'longitude']
cols_listings_cont = ['host_listings_count', 'accommodates']

calendar_money_df = calendar_df[['listing_id', 'date', 'price']]
calendar_money_df = calendar_money_df[-calendar_money_df['price'].isnull()]

listings_money_cols = ['id'] + cols_listings_geo + cols_listings_cont + cols_listings_bool + cols_listings_cat + ['weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'guests_included']

listings_money_df = listings_df[listings_money_cols]
listings_money_df = listings_money_df[listings_money_df['guests_included'] > 0]

moneyloc_df = pd.merge(calendar_money_df, listings_money_df, how='inner', left_on=['listing_id'], right_on=['id'])
moneyloc_df['per_person_price'] = moneyloc_df['price'] / moneyloc_df['guests_included']
moneyloc_df['log_per_person_price'] = np.log(moneyloc_df['per_person_price'])

We indeed have location data for every listing, though recall it is not accurate for about 12%.

### 1.3 Plot the NYC Data

GeoPandas is **AMAZING**! It automatically displays even the polygon! I **love** technology! And in fact, according to the documnetation, it can even compute intersection and difference between two polygons. And it has area methods... whoa...

In [ ]:
boros_gdf['geometry'][0]

Plotting the whole GeoDataFrame is not hard.

In [ ]:
boros_gdf.plot()

In [ ]:
boros_gdf.crs

I need to read about coordinate systems. What controls the x and y axes?

For some reason, I know that converting to espg:4326 (what we read with the neighborhoods geojson file), the x and y axes are exactly lat and long, which means I can place points (as polygons) on the map.

Later, with the Gaussian processes, I will need to clip the predictions to land.

In [ ]:
listings_df.ix[:100, 'latitude'].max()

In [ ]:
plt.scatter(
    listings_df.ix[:100, 'longitude'],
    listings_df.ix[:100, 'latitude'],
    s=2,
    marker='.')

In [ ]:
boros_epsg_gdf = boros_gdf.to_crs(epsg=4326)
boros_ax = boros_epsg_gdf.plot(alpha=0)
boros_ax.hold('on')
boros_ax.scatter(
    listings_df.ix[:, 'longitude'],
    listings_df.ix[:, 'latitude'],
    s=2,
    marker='.'
)

# It is always possible to resize the figure after the fact.

plt.gcf().set_size_inches(15, 15)
plt.title('Scatterplot of AirBnb listing across Boros')

In [ ]:
neighborhoods_ax = neighbourhoods_gdf.plot(alpha=0)
neighborhoods_ax.hold('on')
neighborhoods_ax.scatter(
    listings_df.ix[:, 'longitude'],
    listings_df.ix[:, 'latitude'],
    s=2,
    marker='.'
)

plt.gcf().set_size_inches(15, 15)
plt.title('Scatterplot of AirBnb listing across neighborhoods')

In [ ]:
moneyloc_df

## 2. Figure out how to use GPy

Only plot the data for one day right now.

We fit two models: one to all of the variables we think could be predictive (which we have listed above), and another to just lat and long. The lat-long model is then suitable to plotting, while the full model can be used for prediction.

In [ ]:
import patsy

moneyloc_oneday_df = moneyloc_df[moneyloc_df['date'] == moneyloc_df['date'].min()]


loc_formula  = 'log_per_person_price ~ 1 + longitude + latitude'
full_formula = 'log_per_person_price ~ 1 + longitude + latitude + host_listings_count + instant_bookable + host_identity_verified + host_is_superhost + host_has_profile_pic + C(property_type) + C(room_type) + C(bed_type) + C(cancellation_policy)'

#moneyloc_oneday_scaled_df = moneyloc_oneday_df.copy()

#min_long = moneyloc_oneday_scaled_df['longitude'].min()
#min_lat = moneyloc_oneday_scaled_df['latitude'].min()
#range_long = moneyloc_oneday_scaled_df['longitude'].max() - min_long
#range_lat  = moneyloc_oneday_scaled_df['latitude'].max() - min_lat

# 
#moneyloc_oneday_scaled_df['longitude'] = (moneyloc_oneday_scaled_df['longitude'] - min_long) / range_long
#moneyloc_oneday_scaled_df['latitude'] = (moneyloc_oneday_scaled_df['latitude'] - min_lat) / range_lat

loc_dmat_logY, loc_dmat_X   = patsy.dmatrices(loc_formula, moneyloc_oneday_df)
full_dmat_logY, full_dmat_X = patsy.dmatrices(full_formula, moneyloc_oneday_df)

Log-transforming the Y variable is necessary due to its heavy tail.

In [ ]:
plt.hist(np.ravel(loc_dmat_logY), bins=100)
None

Even a simple linear model will capture much of the variation. Thus, we should include a linear kernel in our GP model.

In [ ]:
loc_lm = sm.OLS(loc_dmat_logY, loc_dmat_X)
loc_lm_results = loc_lm.fit()
loc_lm_results.summary()

In [ ]:
sns.coefplot(loc_formula, moneyloc_oneday_df)
ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=90)

In [ ]:
full_lm = sm.OLS(full_dmat_logY, full_dmat_X)
full_lm_results = full_lm.fit()
full_lm_results.summary()

In [ ]:
sns.coefplot(full_formula, moneyloc_oneday_df)
ax = plt.gca()
plt.setp(ax.get_xticklabels(), rotation=90)

The fit is statistically significant, but has low $R^2$.

In [ ]:
plt.hist(np.ravel(loc_dmat_logY))

In [ ]:
boros_oneday_ax = boros_epsg_gdf.plot(alpha=0)
boros_oneday_ax.hold('on')
boros_oneday_ax.scatter(
    moneyloc_oneday_df.ix[:, 'longitude'],
    moneyloc_oneday_df.ix[:, 'latitude'],
    s=2,
    marker='.'
)

# It is always possible to resize the figure after the fact.

plt.gcf().set_size_inches(15, 15)
plt.title('Scatterplot of AirBnb listing across Boros, on first day')

To get a crude estimate of the noise variance, we fit a linear regression on latitude and longitude.

In [ ]:
# Copied from the example

# CONCLUSION: Choose linear + Matern.

def sparse_GP_regression_2D(X, Y, num_inducing=50, max_iters=100, optimize=True, plot=True):
    """Run a 2D example of a sparse GP regression."""
#    np.random.seed(1234)

    # construct kernel
    n_cols = X.shape[1]
    kern = GPy.kern.Linear(n_cols).add(GPy.kern.Matern32(n_cols))
#    rbf = GPy.kern.Linear(2).add(GPy.kern.RatQuad(2))


    # create simple GP Model
    
    # DEBUG: Make fake data
#    Y = np.sin(X[:, 0:1]) * np.sin(X[:, 1:2]) + np.random.randn(len(Y), 1) * 0.05
    
#    m = GPy.models.SparseGPRegression(X, Y, kernel=rbf, num_inducing=num_inducing)
#    m = GPy.models.GPRegression(X, Y, kernel=rbf, noise_var=10)
#    m = GPy.models.GPRegression(X, np.log(Y), kernel=rbf, noise_var=1)
    m = GPy.models.GPRegression(X, Y, kernel=kern, noise_var=1)

    # contrain all parameters to be positive (but not inducing inputs)
#    m['.*len'] = 2.

    m.checkgrad()

    # optimize
    if optimize:
        m.optimize('tnc', messages=1, max_iters=max_iters)
#        m.optimize(messages=1, max_iters=max_iters)
        

    # plot
    if plot:
        try:
            m.plot()
        except Exception as e:
            print "Unable to plot because ", e

    print(m)
    return m


loc_model_oneday  = sparse_GP_regression_2D(loc_dmat_X, loc_dmat_logY, plot=True)
full_model_oneday = sparse_GP_regression_2D(full_dmat_X, full_dmat_logY, plot=True)

In [ ]:
boros_oneday_ax = boros_epsg_gdf.plot(colormap='binary', alpha=0)
boros_oneday_ax.hold('on')
model_oneday.plot(ax=boros_oneday_ax)
plt.gcf().set_size_inches(15, 15)
plt.title('Model + Data plot AirBnb listing across Boros, on first day')

### 2.2 Quantitatively evaluate predictions

In [ ]:
def eval_model_fit(model, X, Y):
    Yhat, Yhatvar = model.predict(X)
    resids = Yhat - Y
    mse_resids = np.mean(np.square(resids))
    mse_total = np.mean(np.square(Y - np.mean(Y)))

    return {
        'rmse': np.sqrt(mse_resids),
        'Rsq': 1 - mse_resids / mse_total        
    }

In [ ]:
import pprint
pprint.pprint(eval_model_fit(loc_model_oneday, loc_dmat_X, loc_dmat_logY))
pprint.pprint(eval_model_fit(full_model_oneday, full_dmat_X, full_dmat_logY))


In [ ]:
Y.min()